In [1]:
import re
import subprocess
import os
import json

In [2]:
path = '/Users/user/apps-files/github/cyber_completitions/text-files/'
os.chdir(path)

In [3]:
parsed_commands = []

chapters_to_include = ['Available Commands:', 'Usage:', 'Aliases:', 'Flags:', 'Global Flags:', 'Example:']

In [4]:
def get_cli_output(command: str):

    _args = command.split()
    _args.append('-h')

    _out = subprocess.run(_args, capture_output=True)
    _text = _out.stdout.decode("utf-8") + _out.stderr.decode("utf-8")

    with open(command + ".txt", 'w') as file:
        file.write(_text)

    _lines = _text.split("\n")
    return _lines, command

_lines, command = get_cli_output('cyber')
_lines


['Bostrom Bootloader Hub',
 '',
 'Usage:',
 '  cyber [command]',
 '',
 'Available Commands:',
 '  add-genesis-account Add a genesis account to genesis.json',
 '  collect-gentxs      Collect genesis txs and output a genesis.json file',
 '  config              Create or query an application CLI configuration file',
 '  debug               Tool for helping with debugging your application',
 '  export              Export state to JSON',
 '  gentx               Generate a genesis tx carrying a self delegation',
 '  help                Help about any command',
 '  init                Initialize private validator, p2p, genesis, and application configuration files',
 "  keys                Manage your application's keys",
 '  migrate             Migrate genesis to a specified target version',
 '  query               Querying subcommands',
 '  rollback            rollback cosmos-sdk and tendermint state by one height',
 '  start               Run the full node',
 '  status              Query re

In [5]:
get_cli_output('cyber query evidence')

(["Error: invalid evidence hash: encoding/hex: invalid byte: U+002D '-'",
  'Usage:',
  '  cyber query evidence [flags]',
  '',
  'Flags:',
  '      --count-total       count total number of records in evidence to query for',
  '      --height int        Use a specific height to query state at (this can error if the node is pruning state)',
  '  -h, --help              help for evidence',
  '      --limit uint        pagination limit of evidence to query for (default 100)',
  '      --node string       <host>:<port> to Tendermint RPC interface for this chain (default "tcp://localhost:26657")',
  '      --offset uint       pagination offset of evidence to query for',
  '  -o, --output string     Output format (text|json) (default "text")',
  '      --page uint         pagination page of evidence to query for. This sets offset to a multiple of limit (default 1)',
  '      --page-key string   pagination page-key of evidence to query for',
  '      --reverse           results are sorted in

In [6]:
def parse_chapters(_lines: list, command):

    chapters = {}
    chapters['command'] = command
    chapter = "Title"
    chapters[chapter] = []

    for l in _lines:
        if l != "":
            if l in chapters_to_include:
                chapter = l[:-1]
                chapters[chapter] = []
            else:
                chapters[chapter].append(l)


    return chapters

chapters = parse_chapters(_lines, command)
chapters




{'command': 'cyber',
 'Title': ['Bostrom Bootloader Hub'],
 'Usage': ['  cyber [command]'],
 'Available Commands': ['  add-genesis-account Add a genesis account to genesis.json',
  '  collect-gentxs      Collect genesis txs and output a genesis.json file',
  '  config              Create or query an application CLI configuration file',
  '  debug               Tool for helping with debugging your application',
  '  export              Export state to JSON',
  '  gentx               Generate a genesis tx carrying a self delegation',
  '  help                Help about any command',
  '  init                Initialize private validator, p2p, genesis, and application configuration files',
  "  keys                Manage your application's keys",
  '  migrate             Migrate genesis to a specified target version',
  '  query               Querying subcommands',
  '  rollback            rollback cosmos-sdk and tendermint state by one height',
  '  start               Run the full node',

In [7]:

def parse_avail_commands(chapters):
    _temp_commands = []
    
    if 'Available Commands' in chapters:

        for l in chapters['Available Commands']:
            if _match:= re.search(r'^\s+(.*?)\s+(.*)', l):
                _temp_commands.append(_match.group(1))

        chapters['sub_commands'] = _temp_commands
        del chapters['Available Commands']
    return chapters

chapters = parse_avail_commands(chapters)
chapters


{'command': 'cyber',
 'Title': ['Bostrom Bootloader Hub'],
 'Usage': ['  cyber [command]'],
 'Flags': ['  -h, --help                help for cyber',
  '      --home string         directory for config and data (default "/Users/user/.cyber")',
  '      --log_format string   The logging format (json|plain) (default "plain")',
  '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
  '      --trace               print out full stack trace on errors',
  'Use "cyber [command] --help" for more information about a command.'],
 'sub_commands': ['add-genesis-account',
  'collect-gentxs',
  'config',
  'debug',
  'export',
  'gentx',
  'help',
  'init',
  'keys',
  'migrate',
  'query',
  'rollback',
  'start',
  'status',
  'tendermint',
  'testnet',
  'tx',
  'validate-genesis',
  'version']}

In [8]:
def query_and_parse(command):
    _lines, command = get_cli_output(command)
    commands_dict = parse_chapters(_lines, command)
    commands_dict = parse_avail_commands(commands_dict)
    return commands_dict

c0 = [query_and_parse("cyber")]
c0

[{'command': 'cyber',
  'Title': ['Bostrom Bootloader Hub'],
  'Usage': ['  cyber [command]'],
  'Flags': ['  -h, --help                help for cyber',
   '      --home string         directory for config and data (default "/Users/user/.cyber")',
   '      --log_format string   The logging format (json|plain) (default "plain")',
   '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
   '      --trace               print out full stack trace on errors',
   'Use "cyber [command] --help" for more information about a command.'],
  'sub_commands': ['add-genesis-account',
   'collect-gentxs',
   'config',
   'debug',
   'export',
   'gentx',
   'help',
   'init',
   'keys',
   'migrate',
   'query',
   'rollback',
   'start',
   'status',
   'tendermint',
   'testnet',
   'tx',
   'validate-genesis',
   'version']}]

In [9]:
def get_new_commands(chapters: dict):
    if 'sub_commands' in chapters:

        new_commands = []

        for i in chapters['sub_commands']:
            command_candidate = " ".join((chapters['command'] + " " + i).split())
            if command_candidate not in parsed_commands:
                new_commands.append(command_candidate)
                parsed_commands.append(command_candidate)

        return new_commands

    else:
        return None


In [10]:
new_commands = []
for c in c0:
    t = get_new_commands(c)
    if t != None:
        new_commands.extend(t)

new_commands

['cyber add-genesis-account',
 'cyber collect-gentxs',
 'cyber config',
 'cyber debug',
 'cyber export',
 'cyber gentx',
 'cyber help',
 'cyber init',
 'cyber keys',
 'cyber migrate',
 'cyber query',
 'cyber rollback',
 'cyber start',
 'cyber status',
 'cyber tendermint',
 'cyber testnet',
 'cyber tx',
 'cyber validate-genesis',
 'cyber version']

In [11]:
c1 = [ query_and_parse(i) for i in new_commands ]
c1

[{'command': 'cyber add-genesis-account',
  'Title': ['Add a genesis account to genesis.json. The provided account must specify',
   'the account address or key name and a list of initial coins. If a key name is given,',
   'the address will be looked up in the local Keybase. The list of initial tokens must',
   'contain valid denominations. Accounts may optionally be supplied with vesting parameters.'],
  'Usage': ['  cyber add-genesis-account [address_or_key_name] [coin][,[coin]] [flags]'],
  'Flags': ['      --height int               Use a specific height to query state at (this can error if the node is pruning state)',
   '  -h, --help                     help for add-genesis-account',
   '      --keyring-backend string   Select keyring\'s backend (os|file|kwallet|pass|test) (default "os")',
   '      --node string              <host>:<port> to Tendermint RPC interface for this chain (default "tcp://localhost:26657")',
   '  -o, --output string            Output format (text|json)

In [12]:
new_commands = []
for c in c1:
    t = get_new_commands(c)
    if t != None:
        new_commands.extend(t)

new_commands

['cyber debug addr',
 'cyber debug pubkey',
 'cyber debug raw-bytes',
 'cyber keys add',
 'cyber keys delete',
 'cyber keys export',
 'cyber keys import',
 'cyber keys list',
 'cyber keys migrate',
 'cyber keys mnemonic',
 'cyber keys parse',
 'cyber keys show',
 'cyber query account',
 'cyber query auth',
 'cyber query authz',
 'cyber query bandwidth',
 'cyber query bank',
 'cyber query block',
 'cyber query distribution',
 'cyber query dmn',
 'cyber query evidence',
 'cyber query feegrant',
 'cyber query gov',
 'cyber query graph',
 'cyber query grid',
 'cyber query ibc',
 'cyber query ibc-transfer',
 'cyber query liquidity',
 'cyber query mint',
 'cyber query params',
 'cyber query rank',
 'cyber query resources',
 'cyber query slashing',
 'cyber query staking',
 'cyber query tendermint-validator-set',
 'cyber query tx',
 'cyber query txs',
 'cyber query upgrade',
 'cyber query wasm',
 'cyber tendermint reset-state',
 'cyber tendermint show-address',
 'cyber tendermint show-node-id'

In [13]:
c2 = [ query_and_parse(i) for i in new_commands ]
c2

[{'command': 'cyber debug addr',
  'Title': ['Convert an address between hex encoding and bech32.'],
  'Example': ['$ cyber debug addr cosmos1e0jnq2sun3dzjh8p2xq95kk0expwmd7shwjpfg'],
  'Usage': ['  cyber debug addr [address] [flags]'],
  'Flags': ['  -h, --help   help for addr'],
  'Global Flags': ['      --home string         directory for config and data (default "/Users/user/.cyber")',
   '      --log_format string   The logging format (json|plain) (default "plain")',
   '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
   '      --trace               print out full stack trace on errors']},
 {'command': 'cyber debug pubkey',
  'Title': ["Decode a pubkey from proto JSON and display it's address."],
  'Example': ['$ cyber debug pubkey \'{"@type":"/cosmos.crypto.secp256k1.PubKey","key":"AurroA7jvfPd1AadmmOvWM2rJSwipXfRf8yD6pLbA2DJ"}\''],
  'Usage': ['  cyber debug pubkey [pubkey] [flags]'],
  'Flags': ['  -h, --help   help for

In [14]:
new_commands = []
for c in c2:
    t = get_new_commands(c)
    if t != None:
        new_commands.extend(t)

new_commands

['cyber query auth account',
 'cyber query auth accounts',
 'cyber query auth params',
 'cyber query authz grants',
 'cyber query authz grants-by-grantee',
 'cyber query authz grants-by-granter',
 'cyber query bandwidth load',
 'cyber query bandwidth neuron',
 'cyber query bandwidth params',
 'cyber query bandwidth price',
 'cyber query bandwidth total',
 'cyber query bank balances',
 'cyber query bank denom-metadata',
 'cyber query bank total',
 'cyber query distribution commission',
 'cyber query distribution community-pool',
 'cyber query distribution params',
 'cyber query distribution rewards',
 'cyber query distribution slashes',
 'cyber query distribution validator-outstanding-rewards',
 'cyber query dmn params',
 'cyber query dmn thought',
 'cyber query dmn thought-stats',
 'cyber query dmn thoughts',
 'cyber query dmn thoughts-stats',
 'cyber query feegrant grant',
 'cyber query feegrant grants-by-grantee',
 'cyber query feegrant grants-by-granter',
 'cyber query gov deposit',

In [15]:
c3 = [ query_and_parse(i) for i in new_commands ]
c3

[{'command': 'cyber query auth account',
  'Title': ['Query for account by address'],
  'Usage': ['  cyber query auth account [address] [flags]'],
  'Flags': ['      --height int      Use a specific height to query state at (this can error if the node is pruning state)',
   '  -h, --help            help for account',
   '      --node string     <host>:<port> to Tendermint RPC interface for this chain (default "tcp://localhost:26657")',
   '  -o, --output string   Output format (text|json) (default "text")'],
  'Global Flags': ['      --chain-id string     The network chain ID',
   '      --home string         directory for config and data (default "/Users/user/.cyber")',
   '      --log_format string   The logging format (json|plain) (default "plain")',
   '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
   '      --trace               print out full stack trace on errors']},
 {'command': 'cyber query auth accounts',
  'Title':

In [16]:
new_commands = []
for c in c3:
    t = get_new_commands(c)
    if t != None:
        new_commands.extend(t)

new_commands

['cyber query ibc channel channels',
 'cyber query ibc channel client-state',
 'cyber query ibc channel connections',
 'cyber query ibc channel end',
 'cyber query ibc channel next-sequence-receive',
 'cyber query ibc channel packet-ack',
 'cyber query ibc channel packet-commitment',
 'cyber query ibc channel packet-commitments',
 'cyber query ibc channel packet-receipt',
 'cyber query ibc channel unreceived-acks',
 'cyber query ibc channel unreceived-packets',
 'cyber query ibc client consensus-state',
 'cyber query ibc client consensus-states',
 'cyber query ibc client header',
 'cyber query ibc client params',
 'cyber query ibc client self-consensus-state',
 'cyber query ibc client state',
 'cyber query ibc client states',
 'cyber query ibc client status',
 'cyber query ibc connection connections',
 'cyber query ibc connection end',
 'cyber query ibc connection path',
 'cyber query wasm contract-state all',
 'cyber query wasm contract-state raw',
 'cyber query wasm contract-state sm

In [17]:
c4 = [ query_and_parse(i) for i in new_commands ]
c4

[{'command': 'cyber query ibc channel channels',
  'Title': ['Query all channels from a chain'],
  'Usage': ['  cyber query ibc channel channels [flags]',
   'Examples:',
   'cyber query ibc channel channels'],
  'Flags': ['      --count-total       count total number of records in channels to query for',
   '      --height int        Use a specific height to query state at (this can error if the node is pruning state)',
   '  -h, --help              help for channels',
   '      --limit uint        pagination limit of channels to query for (default 100)',
   '      --node string       <host>:<port> to Tendermint RPC interface for this chain (default "tcp://localhost:26657")',
   '      --offset uint       pagination offset of channels to query for',
   '  -o, --output string     Output format (text|json) (default "text")',
   '      --page uint         pagination page of channels to query for. This sets offset to a multiple of limit (default 1)',
   '      --page-key string   paginati

In [18]:
new_commands = []
for c in c4:
    t = get_new_commands(c)
    if t != None:
        new_commands.extend(t)

new_commands

[]

In [19]:
all_commands = c0 + c1 + c2 + c3 + c4
all_commands

[{'command': 'cyber',
  'Title': ['Bostrom Bootloader Hub'],
  'Usage': ['  cyber [command]'],
  'Flags': ['  -h, --help                help for cyber',
   '      --home string         directory for config and data (default "/Users/user/.cyber")',
   '      --log_format string   The logging format (json|plain) (default "plain")',
   '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
   '      --trace               print out full stack trace on errors',
   'Use "cyber [command] --help" for more information about a command.'],
  'sub_commands': ['add-genesis-account',
   'collect-gentxs',
   'config',
   'debug',
   'export',
   'gentx',
   'help',
   'init',
   'keys',
   'migrate',
   'query',
   'rollback',
   'start',
   'status',
   'tendermint',
   'testnet',
   'tx',
   'validate-genesis',
   'version']},
 {'command': 'cyber add-genesis-account',
  'Title': ['Add a genesis account to genesis.json. The provided account must s

In [ ]:
# n = 0
# for i in all_commands:
#     if i['command'] == 'cyber query evidence':
#         print(n)

#     n += 1

# del all_commands[40]

In [ ]:
for i in all_commands[:5]:
    print (re.match(r'\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[\w-]+))\s+(?:<(?P<format>.*)>)?\s*(?P<description>.*)?', i['Flags'][0]).group(0))

In [ ]:

count = 0
for i in all_commands:

    if i['Flags'] != []:
        i['flags_parsed'] = []
        for b in i['Flags']:
            if match1:=re.match(r'\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[.\w-]+))\s(?:(?P<format>(uint32|ints|strings|int 64Slice|int|string|uint|float|bytesHex)))?\s*(?P<description>.*)?', b):
                i['flags_parsed'].append((match1.group('long'), match1.group('short'), match1.group('format'), match1.group('description'),))

In [ ]:
for c in all_commands:
    print(f"\n# {c['title']}\nextern '{c['command']}' [")
    for f in c['flags_parsed']:
        if "." not in f[0]:
            print(f"\t--{f[0]}{'(-'+f[1]+')' if f[1] else ''}\t\t# {f[3]}")
    print("]")

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(all_commands).sort_values('command', ascending=False)

In [ ]:
path = "/Users/user/apps-files/github/cyber_completitions/json-files/cyber-query distribution.json"

with open(path, "r") as f:
    content = f.read()

In [ ]:
import pandas as pd

pd.DataFrame.from_records(json.loads(content))

In [ ]:
_list = []

for _file in os.scandir("/Users/user/apps-files/github/cyber_completitions/json-files/"):
    with open(_file, "r") as f:
        content = f.read()
        _list.append(json.loads(content))

In [ ]:
[i for i in _list if i != []]

In [ ]:
df = pd.concat([pd.DataFrame.from_records(i) for i in _list if i != []])
df

In [ ]:
df['com1'] = "cyber " + df['_command_root'] + " " + df['_command']
df

In [ ]:
_l2 = df.loc[:, ['_description', 'com1']].to_dict('records')
_l2

In [ ]:
for l in _l2:
    print(f'# {l["_description"]}\nextern "{l["com1"]}" [] \n')

In [ ]:
for i in json.loads(content):
    print("\t", i['_command'], "\t\t#", i['_description'])